# Activations to Visualization

This notebook contains code for loading and visualizing activations of an LLM during output generation. The code assumes the generated output is stored in a saved .pt file on your google drive. 


Currently, the model being tested is [Alpaca-LoRA](https://github.com/tloen/alpaca-lora/), 7B parameters

## Description of the saved file

The input prompt, generated output and hidden states are saved as a `.pt` file from pytorch. 

The file is saved as `{input_prompt}.pt`

To load the file use,

`data = torch.load("{input_prompt}.pt", map_location=torch.device('cpu'))`

A peek into what that file looks like when loaded:
```
prompt = data['prompt']
hidden_states = data['hidden_states']
output_sequence = data['sequences'][0]
output = data['output'].split("Response:")[1]
```

The shape of the hidden states will be:

```
hidden states for full output shape: (n_output_tokens, n_layers, num_beams, n_iterations, hidden_size)

n_output_tokens : includes the input tokens, I think even in input each token is fed one at a time
n_layers : 33, number of decoder layers + input layer
num_beams : 1, number of beam searches
n_iterations: n_input_tokens, for first and then 1 for all other output tokens
hidden_size: 4096, based on model config
```

### Mount Google Drive

In [2]:
from google.colab import drive

drive.mount('/content/drive')
!ls '/content/drive/MyDrive/llm'

Mounted at /content/drive
activations  models  visualizations


###Import Packages

In [3]:
!pip install -q git+https://github.com/wazeerzulfikar/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm
import torch
from tqdm import tqdm 
import transformers

### Load Activations (.pt file)

In [5]:
input_prompt = "What_is_4_+_2?"

activation_path = "/content/drive/MyDrive/llm/activations/"+str(input_prompt)+".pt"
data = torch.load(activation_path, map_location=torch.device('cpu'))

# from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
# tokenizer = LlamaTokenizer.from_pretrained('decapoda-research/llama-7b-hf')

###Define Visualization Options & Utility Functions


In [25]:
def generate_bitmap_animation(data):

  hidden_states = data['hidden_states']
  output_response = data['output'].split("Response:")[1]
  # tokenized_output_response = tokenizer.encode(output_response)

  all_images = []
  vmin=0
  vmax=0

  #iterate through hidden states of all tokens 
  for token_id, token_hidden_states in tqdm(enumerate(hidden_states)):
    # Disregard first hidden state (which inclued all input? tokens)
    if token_id > 0:
      # current_token = tokenized_output_response[token_id]
      current_token_string = output_response[token_id]
      # print("Token ID: "+str(token_id)+"\t Token Value: " + str(current_token), "\t Token String: '" + str(current_token_string)+"'")

      # Initialize an empty dictionary to store activations
      activations = []

      # iterate through all layers for each token's hidden states
      for layer_id, layers in enumerate(token_hidden_states):
        # print("Layer: "+str(layer_id))
        for beam_id, beams in enumerate(layers):
          # print("Beam: "+str(beam_id))
          for token_activation_id, token_activations in enumerate(beams):
            # print(token_activations.shape)
            token_activations_np = token_activations.numpy()  # Detach and convert to NumPy array
            activations.extend(token_activations_np)

      # Determine the size of the square image
      image_size = int(np.ceil(np.sqrt(len(activations))))

      # Create an empty square image with pixel values set to zero
      img = np.zeros((image_size, image_size), dtype=np.uint8)

      # Fill in the image with the normalized activation values
      img.flat[:len(activations)] = activations

      # Add the image to the all_images list
      all_images.append(img)

      # Update the global vmin and vmax
      vmin_token = np.min(activations)
      if vmin_token < vmin:
        vmin = vmin_token
      vmax_token = np.max(activations)
      if vmax_token > vmax:
        vmax = vmax_token

  #Set log norm based on global max and min activations values
  log_norm = LogNorm(vmin=vmin, vmax=vmax)

  # Set the matplotlib backend to save files in the desired format
  matplotlib.use("Agg")

  # Define the update function for the animation
  def update(frame):
      plt.clf()
      plt.imshow(all_images[frame], cmap='viridis', norm=log_norm)
      plt.title(f"Token {frame + 1}: {output_response[frame]}", fontsize=30)
      plt.axis('off')

  # Create the animation
  fig = plt.figure(figsize=(20, 20))
  ani = FuncAnimation(fig, update, frames=len(output_response)-1, interval=250)

  # Save the animation as an MP4 file
  output_file = data["prompt"].replace(' ', '_')+".mp4"
  output_path = "/content/drive/MyDrive/llm/visualizations/"+str(output_file)
  ani.save(output_path, dpi=100, writer="ffmpeg")

  # Set the matplotlib backend back to the default
  matplotlib.use("module://ipykernel.pylab.backend_inline")

  print("visualization succesfully saved: '"+str(output_path)+"'")

def generate_visuals(data, visual_type="bitmap_animation"):
  if visual_type == "bitmap_animation":
    generate_bitmap_animation(data)

###Run Visual Generation & Save Ouput

In [26]:
generate_visuals(data, visual_type="bitmap_animation")

10it [00:00, 27.98it/s]


visualization succesfully saved: '/content/drive/MyDrive/llm/visualizations/What_is_4_+_2?.mp4'


## UNDER DEVELOPMENT - Visualize Reduced Dim

In [37]:
%pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=1b63fde4a6c9d64d284ac102998e20c8413d9596e0e0513a68cb0b318f5f0700
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55640 sha256=e59073f4876b034c8d1c22cba6a2779cb1c05f35ff80d8e7e9bcbc587a3703d2
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [51]:
import os
import pandas as pd
import umap
import matplotlib
from tqdm import tqdm
import seaborn as sns

activation_folder_list = os.listdir("/content/drive/MyDrive/llm/test/")

all_prompt_activations = []
all_prompt_filenames = []

count = 0
for activation_file in tqdm(activation_folder_list):
  if count < 1000:
    print(activation_file)

    filename = '_'.join(activation_file.split('_')[-3:])
    all_prompt_filenames.append(filename)

    data = torch.load(activation_path, map_location=torch.device('cpu'))

    hidden_states = data['hidden_states']
    output_response = data['output'].split("Response:")[1]
    # tokenized_output_response = tokenizer.encode(output_response)

    all_activations = []
    vmin=0
    vmax=0

    #iterate through hidden states of all tokens 
    for token_id, token_hidden_states in tqdm(enumerate(hidden_states)):
      # Disregard first hidden state (which inclued all input? tokens)
      if token_id > 0:
        # current_token = tokenized_output_response[token_id]
        current_token_string = output_response[token_id]
        # print("Token ID: "+str(token_id)+"\t Token Value: " + str(current_token), "\t Token String: '" + str(current_token_string)+"'")

        # Initialize an empty dictionary to store activations
        activations = []

        # iterate through all layers for each token's hidden states
        for layer_id, layers in enumerate(token_hidden_states):
          # print("Layer: "+str(layer_id))
          for beam_id, beams in enumerate(layers):
            # print("Beam: "+str(beam_id))
            for token_activation_id, token_activations in enumerate(beams):
              # print(token_activations.shape)
              token_activations_np = token_activations.numpy()  # Detach and convert to NumPy array
              activations.extend(token_activations_np)

        # all_activations.append(activations)
        all_activations.extend(activations)
    all_prompt_activations.append(all_activations)

    count += 1

# from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
# tokenizer = LlamaTokenizer.from_pretrained('decapoda-research/llama-7b-hf')

  0%|          | 0/1132 [00:00<?, ?it/s]

Question_131_character_qa_open.pt



10it [00:00, 127.42it/s]
  0%|          | 1/1132 [00:00<02:21,  7.97it/s]

Question_132_character_qa_open.pt



10it [00:00, 114.91it/s]
  0%|          | 2/1132 [00:00<02:29,  7.54it/s]

...


10it [00:00, 110.16it/s]
 88%|████████▊ | 998/1132 [02:54<00:22,  6.07it/s]

meta-data.csv



10it [00:00, 100.47it/s]
 88%|████████▊ | 999/1132 [02:54<00:21,  6.21it/s]

Question_1129_odd_one_out.pt



10it [00:00, 114.79it/s]
100%|██████████| 1132/1132 [02:54<00:00,  6.48it/s]


In [52]:
d = {'prompt_filenames': all_prompt_filenames, 'prompt_activations': all_prompt_activations}
df = pd.DataFrame(data=d)

In [53]:
df.head()

,prompt_filenames,prompt_activations
0,character_qa_open.pt,"[0.0003011, -0.0008216, -4.98e-05, 0.001191, 0..."
1,character_qa_open.pt,"[0.0003011, -0.0008216, -4.98e-05, 0.001191, 0..."
2,character_qa_open.pt,"[0.0003011, -0.0008216, -4.98e-05, 0.001191, 0..."
3,character_qa_open.pt,"[0.0003011, -0.0008216, -4.98e-05, 0.001191, 0..."
4,character_qa_open.pt,"[0.0003011, -0.0008216, -4.98e-05, 0.001191, 0..."


In [ ]:
reducer = umap.UMAP()

# Prepare the input for UMAP
activations = np.stack(df['prompt_activations'].to_numpy())

# Apply UMAP to compress dimensions to 2D
reducer = umap.UMAP(n_components=2, n_neighbors=2)
umap_result = reducer.fit_transform(activations)

In [ ]:
#Define coloring algo
def generate_color_map(unique_elements):
    color_palette = sns.color_palette('husl', len(unique_elements))
    color_map = {key: color for key, color in zip(unique_elements, color_palette)}
    return color_map

# Create a scatter plot using the reduced dimensions
unique_filenames = df['prompt_filenames'].unique()
color_map = generate_color_map(unique_filenames)

for i, filename in enumerate(df['prompt_filenames']):
    plt.scatter(umap_result[i, 0], umap_result[i, 1], color=color_map[filename], label=filename)

plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.title('UMAP Visualization of Prompt Activations')

# Add legend and remove duplicates
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.show()